In [1]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel, PeftConfig
from trl import SFTTrainer
import time
from dotenv import load_dotenv
import random
random.seed(42)

load_dotenv()

base_model = "meta-llama/Llama-2-7b-chat-hf"
new_model = "Llama-2-7b-chat-hf-text2sql"
token = os.getenv('token')

/Users/leo/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(base_model, token = token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model = AutoModelForCausalLM.from_pretrained(base_model, 
                                            #quantization_config=quant_config,
                                             token = token)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:03<00:00,  1.62s/it]


In [3]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)

In [4]:
prompt = "[INST]<<SYS>>\nBelow are sql tables schemas paired with instruction that describes a task. \n                Using valid SQLite, write a response that appropriately completes the request \n                for the provided tables. <</SYS>>\n\nWhat was round 7's lowest overall?CREATE TABLE table_name_72 (\n    overall INTEGER,\n    round VARCHAR\n)[/INST]"
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

[INST]<<SYS>>
Below are sql tables schemas paired with instruction that describes a task. 
                Using valid SQLite, write a response that appropriately completes the request 
                for the provided tables. <</SYS>>

What was round 7's lowest overall?CREATE TABLE table_name_72 (
    overall INTEGER,
    round VARCHAR
)[/INST]  To complete the task of finding the lowest overall score from round 7, we need to query the `table_name_72` table. The `overall` column in this table represents the score for each round, and the `round` column indicates which round it belongs to.

Here's the query to retrieve the lowest overall score from round 7:
```sql
SELECT MIN(overall) FROM table_name_72 WHERE round = '7';
```
This query will return the minimum value in the `overall` column for all rows where the `round` column is equal to '7'.


In [5]:
del model
del tokenizer
del pipe

In [6]:
peft_params = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model_id = new_model
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, 
                                             return_dict=True, 
                                             #load_in_8bit=True, 
                                             #device_map='auto'
                                            )
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
peft_model = PeftModel.from_pretrained(model, peft_model_id)
del model

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:03<00:00,  1.52s/it]
/Users/leo/.pyenv/versions/3.10.13/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [7]:
pipe = pipeline(task="text-generation", model=peft_model, tokenizer=tokenizer, max_length=2000)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

In [8]:
prompt = "[INST]<<SYS>>\nBelow are sql tables schemas paired with instruction that describes a task. \n                Using valid SQLite, write a response that appropriately completes the request \n                for the provided tables. <</SYS>>\n\nWhat was round 7's lowest overall?CREATE TABLE table_name_72 (\n    overall INTEGER,\n    round VARCHAR\n)[/INST]"
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

[INST]<<SYS>>
Below are sql tables schemas paired with instruction that describes a task. 
                Using valid SQLite, write a response that appropriately completes the request 
                for the provided tables. <</SYS>>

What was round 7's lowest overall?CREATE TABLE table_name_72 (
    overall INTEGER,
    round VARCHAR
)[/INST]  To complete the task, we need to write a SQL query that retrieves the lowest overall score from round 7. Based on the provided schema, the query should be:
```sql
SELECT overall FROM table_name_72 WHERE round = '7';
```
This will retrieve the lowest overall score from round 7, which is stored in the `overall` column of the `table_name_72` table.
